In [ ]:
import chipwhisperer as cw

In [ ]:
def capture_ttest(N, picoscope=False, splot=None, zero_cross_only=False):
    """Captures a T-Test"""
    import numpy as np
    from tqdm.autonotebook import trange
    ktp = cw.ktp.TVLATTest()
    ktp.init(N)

    group1 = []
    group2 = []
    g1_i = 0
    g2_i = 0
    for i in trange(2*N, desc='Capturing traces'):
        key, text = ktp.next()  # TVLA T-Test changes PT between two options

        if picoscope:
            ps.runBlock()
        trace = cw.capture_trace(scope, target, text, key)
        
        if trace is None:
            print("Serial Error - dropped trace")
            continue

        if picoscope:
            while ps.isReady() == False:
                continue

            wave = ps.getDataV('A')
        else:
            wave = trace.wave
            
        if i % 25 == 0:
            if splot:
                splot.update(wave) # wave is the name for the data for our power trace
            
        if zero_cross_only:
            wave = np.where(np.diff(np.sign(wave)))[0]

        if trace.textin == bytearray([0xda, 0x39, 0xa3, 0xee, 0x5e, 0x6b, 0x4b, 0x0d, 0x32, 0x55, 0xbf, 0xef, 0x95, 0x60, 0x18, 0x90]):
            group1.append(wave)
            g1_i += 1
        else:
            group2.append(wave)
            g2_i += 1

    group1 = np.array(group1)
    print(len(group1))
    group2 = np.array(group2)
    print(len(group2))
    
    return (group1, group2)

In [ ]:
def plot_t(t_val, N, titledata=""):
    import matplotlib.pylab as plt
    ldata = len(t_val)
    plt.plot(t_val)
    plt.plot([0, ldata], [4.5, 4.5], 'k--')
    plt.plot([0, ldata], [-4.5, -4.5], 'k--')
    plt.title("TVLA Results, N=%d, %s"%(N, titledata))
    plt.xlabel("Sample Number")
    plt.ylabel("T-Test Value")

In [ ]:
import os

def npload(filename):
    return np.load(os.path.join(data_dir, filename), allow_pickle=True)

def nptload(prefix):
    group1 = npload(prefix + "_tvla_group1.npy")
    group2 = npload(prefix + "_tvla_group2.npy")
    N = round((len(group1) + len(group2)) / 2)
    return group1, group2, N

def npsave(filename, array):
    np.save(os.path.join(data_dir, filename), array)
    
def nptsave(prefix, group1, group2):
    npsave(prefix + "_tvla_group1.npy", group1)
    npsave(prefix + "_tvla_group2.npy", group2)
    

In [ ]:
def create_diffs(group, start=0, crossingoffset=1):
    """Finds zero crossing in each trace, returns time between each crossing"""

    grouptime = [ [] ]*len(group)

    for i in range(0, len(group)):
        where = np.where(np.diff(np.sign(group[i][start:])))[0]
        offset = where[crossingoffset]

        where = where[crossingoffset:]
        where = [w-offset for w in where]

        gtime = np.ediff1d(where)
        grouptime[i] = gtime
    
    return grouptime

In [ ]:
def create_diffs_from_crossings(group, start=0, crossingoffset=1):
    """Assuming zero crossing already recorded, returns time between each crossing"""

    grouptime = [ [] ]*len(group)

    for i in range(0, len(group)):
        where = group[i]
        offset = where[crossingoffset]

        where = where[crossingoffset:]
        where = [w-offset for w in where]

        gtime = np.ediff1d(where)
        grouptime[i] = gtime
    
    return grouptime

In [ ]:
from tqdm.notebook import trange
import numpy as np
import time

def capture_cpa(N, picoscope=False, splot=None, zero_cross_only=False, ktp=cw.ktp.Basic()):
    textins = []
    textouts = []
    waves = []
    keys = []

    for i in trange(N, desc='Capturing traces'):
        key, text = ktp.next() # new plaintext, same key
        
        if picoscope:
            ps.runBlock()
        trace = cw.capture_trace(scope, target, text, key) # set key, send plaintext, receive ciphertext, capture power trace
        if not trace:
            continue

        if picoscope:
            while ps.isReady() == False:
                continue

            wave = ps.getDataV('A')
        else:
            wave = trace.wave

        waves.append(wave)
        textins.append(trace.textin)
        textouts.append(trace.textout)
        keys.append(trace.key)

        # Update our plot with a new trace
        if i % 10 == 0 and splot:            
            splot.update(wave) # wave is the name for the data for our power trace
    
    return waves, textins, textouts, keys

In [ ]:
import numpy as np
import zarr
from numcodecs import lz4
from numcodecs import Blosc

def zarr_store(filename, waves, textins, textouts, keys, compress=False):

    if compress is False:
        compressor = None
        compressor_metadata = None
    else:
        compressor = Blosc(cname='lz4hc', clevel=9, shuffle=Blosc.SHUFFLE)
        compressor_metadata = None
    
    zarr_dir = zarr.DirectoryStore(filename)
    zarr_group = zarr.hierarchy.group(store=zarr_dir)
    zarr_group_tile = zarr_group.require_group("0/0")

    samples = waves[0].shape[0]

    zarr_group_tile.zeros(
        name="plaintext",
        shape=(len(textins),16),
        chunks=(len(textins),16),
        dtype=np.uint8,
        compressor=compressor_metadata
    )

    zarr_group_tile.zeros(
        name="key",
        shape=(len(textins),16),
        chunks=(len(textins),16),
        dtype=np.uint8,
        compressor=compressor_metadata
    )

    zarr_group_tile.zeros(
        name="ciphertext",
        shape=(len(textins),16),
        chunks=(len(textins),16),
        dtype=np.uint8,
        compressor=compressor_metadata
    )

    zarr_group_tile.zeros(
        name="traces",
        shape=(len(waves),samples),
        chunks=(len(waves),samples),
        dtype=np.double, # int8 or int16
        compressor=compressor
    )

    zarr_group_tile['traces'] = waves
    zarr_group_tile['plaintext'] = textins
    zarr_group_tile['ciphertext'] = textouts
    zarr_group_tile['key'] = keys

    zarr_dir.close()

    print(zarr_group_tile.tree())

In [ ]:
import shutil

def zarr_delete(filename):
    """Wrapper that just delets a directory blindly"""
    try:
        shutil.rmtree(filename)
    except:
        print("Exception on delete - ignored!")

In [1]:
import estraces

def save_ets(waves, textins, textouts, keys, filename, overwrite=False):
    ths = estraces.read_ths_from_ram(samples=np.array(waves), **{'plaintext':np.array(textins), 'ciphertext':np.array(textouts), 'key':np.array(keys)})
    writer = estraces.ETSWriter(filename, overwrite=overwrite)
    writer.add_trace_header_set(ths)
    writer.close()